In [56]:
import pandas as pd
import numpy as np
from helpers import display_all, to_snake_case

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [3]:
DATA_PATH = "../data/bluebook-for-bulldozers/"

In [6]:
df_raw = pd.read_csv(f'{DATA_PATH}Train.csv', low_memory=False, parse_dates=["saledate"])

In [33]:
df_raw = df_raw.rename(mapper=to_snake_case, axis=1)

In [34]:
df_raw.shape

(401125, 53)

In [35]:
display_all(df_raw.head().T)

,0,1,2,3,4
sales_id,1139246,1139248,1139249,1139251,1139253
sale_price,66000,57000,10000,38500,11000
machine_id,999089,117657,434808,1026470,1057373
model_id,3157,77,7009,332,17311
datasource,121,121,121,121,121
auctioneer_id,3,3,3,3,3
year_made,2004,1996,2001,2001,2007
machine_hours_current_meter,68,4640,2838,3486,722
usage_band,Low,Low,High,High,Medium
saledate,2006-11-16 00:00:00,2004-03-26 00:00:00,2004-02-26 00:00:00,2011-05-19 00:00:00,2009-07-23 00:00:00


In [36]:
display_all(df_raw.describe(include="all").T)

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
sales_id,401125,NaN,NaN,NaN,NaN,NaN,1.91971e+06,909021,1.13925e+06,1.41837e+06,1.63942e+06,2.24271e+06,6.33334e+06
sale_price,401125,NaN,NaN,NaN,NaN,NaN,31099.7,23036.9,4750,14500,24000,40000,142000
machine_id,401125,NaN,NaN,NaN,NaN,NaN,1.2179e+06,440992,0,1.0887e+06,1.27949e+06,1.46807e+06,2.48633e+06
model_id,401125,NaN,NaN,NaN,NaN,NaN,6889.7,6221.78,28,3259,4604,8724,37198
datasource,401125,NaN,NaN,NaN,NaN,NaN,134.666,8.96224,121,132,132,136,172
auctioneer_id,380989,NaN,NaN,NaN,NaN,NaN,6.55604,16.9768,0,1,2,4,99
year_made,401125,NaN,NaN,NaN,NaN,NaN,1899.16,291.797,1000,1985,1995,2000,2013
machine_hours_current_meter,142765,NaN,NaN,NaN,NaN,NaN,3457.96,27590.3,0,0,0,3025,2.4833e+06
usage_band,69639,3,Medium,33985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
saledate,401125,3919,2009-02-16 00:00:00,1932,1989-01-17 00:00:00,2011-12-30 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
df_raw["sale_price"] = np.log(df_raw["sale_price"])

In [65]:
from fastai.tabular import add_datepart

In [67]:
??add_datepart

Signature:
add_datepart(
    df: pandas.core.frame.DataFrame,
    field_name: str,
    prefix: str = None,
    drop: bool = True,
    time: bool = False,
)
Source:   
def add_datepart(df:DataFrame, field_name:str, prefix:str=None, drop:bool=True, time:bool=False):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."
    make_date(df, field_name)
    field = df[field_name]
    prefix = ifnone(prefix, re.sub('[Dd]ate$', '', field_name))
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start', 
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[prefix + n] = getattr(field.dt, n.lower())
    df[prefix + 'Elapsed'] = field.astype(np.int64) // 10 ** 9
    if drop: df.drop(field_name, axis=1, inplace=True)
    return df
File:      ~/.local/share/virtualenvs/fast-ai-ml-for-coders-_QedMMG8/lib/python

In [79]:



add_date_parts(df_raw)

,sales_id,sale_price,machine_id,model_id,datasource,auctioneer_id,year_made,machine_hours_current_meter,usage_band,fi_model_desc,...,saledate_week,saledate_day,saledate_dayofweek,saledate_dayofyear,saledate_is_month_end,saledate_is_month_start,saledate_is_quarter_end,saledate_is_quarter_start,saledate_is_year_end,saledate_is_year_start
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,521D,...,46,16,3,320,False,False,False,False,False,False
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,950FII,...,13,26,4,86,False,False,False,False,False,False
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,226,...,9,26,3,57,False,False,False,False,False,False
3,1139251,10.558414,1026470,332,121,3.0,2001,3486.0,High,PC120-6E,...,20,19,3,139,False,False,False,False,False,False
4,1139253,9.305651,1057373,17311,121,3.0,2007,722.0,Medium,S175,...,30,23,3,204,False,False,False,False,False,False
5,1139255,10.184900,1001274,4605,121,3.0,2004,508.0,Low,310G,...,51,18,3,353,False,False,False,False,False,False
6,1139256,9.952278,772701,1937,121,3.0,1993,11540.0,High,790ELC,...,35,26,3,239,False,False,False,False,False,False
7,1139261,10.203592,902002,3539,121,3.0,2001,4883.0,High,416D,...,46,17,3,321,False,False,False,False,False,False
8,1139272,9.975808,1036251,36003,121,3.0,2008,302.0,Low,430HAG,...,35,27,3,239,False,False,False,False,False,False
9,1139275,11.082143,1016474,3883,121,3.0,1000,20700.0,Medium,988B,...,32,9,3,221,False,False,False,False,False,False
